<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
%%capture
%pip install selenium beautifulsoup4 datasets transformers[torch] deep-translator

## Scraping data from AutoZone

We are going to extract the content from the articles found in AutoZone sitemap.

In [2]:
from articles import extract_articles
import pandas as pd

In [3]:
data_dir = './data'

In [4]:
articles = extract_articles(data_dir)

In [5]:
print(articles[0])

{'title': 'Car AC Blowing Hot Air', 'content': "Understanding the Causes\n\nA quick understanding of how air conditioning works can help with understanding what the causes could be. When AC is turned on, refrigerant that flows through the system absorbs heat from your vehicle's cabin where it's removed and, through a series of parts and processes, the heat is released into the atmosphere before circulating back and repeating the process. There are several points where something can be wrong, causing warm air rather than cool:\n\nThere isn't sufficient airflow in the cabin. This could be a problem with a bad blower motor, but more commonly a plugged cabin air filter is the culprit.\nThere isn't enough refrigerant. The gas that circulates through the system can leak out, preventing it from working efficiently.\nThe compressor may not be cycling. A clutch issue or a compressor failure can prevent the AC system from being able to disperse the heat the refrigerant has absorbed.\nThe expansi

In [18]:
faq_questions = []
for article in articles:
    faq_questions.extend(article['faq_questions'])

In [84]:
df_faq = pd.DataFrame(faq_questions)

In [85]:
has_autozone_text = df_faq['question'].str.contains('AutoZone')
df_faq.drop(index=df_faq[has_autozone_text].index, inplace=True)

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from articles import extract_articles
import pandas as pd

In [ ]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
DATA_NAME = "yahoo_answers_qa"
yahoo_answers_qa = load_dataset(DATA_NAME)
yahoo_answers_qa['train'][9]
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

In [ ]:
# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)


In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result